In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image
import albumentations as A

%matplotlib inline
from IPython.core.pylabtools import figsize

In [ ]:
!ls /Users/joe/Pictures/

In [ ]:
A.Affine?

In [ ]:
data = [
    {"source":"/Users/joe/Pictures/G0040113.JPG", "boxes":[[600, 212, 775, 300], [330, 390, 420, 440]],
    "class":["hov_sign", "sticker"]},
    {"source":"/Users/joe/Pictures/G0061823.JPG", "boxes":[[620, 320, 680, 460], [160, 460, 230, 520]],
    "class":["masonic_thing", "car"]}
]

for i in range(len(data)):
    for j in range(2):
        b = data[i]["boxes"][j]
        data[i]["boxes"][j] = [4*k for k in b]

In [ ]:
tfmlist = [
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Affine(shear=(-20,20), rotate=(-20,20), p=1)
]

In [ ]:
transform = A.Compose(tfmlist, bbox_params=A.BboxParams(format='pascal_voc'))

In [ ]:
def _augment(image, boxes, tfm=None):
    """
    :image: PIL image
    :boxes: list of lists; boxes in [left top right bottom] format
    :tfm: Albumentations transform
    """
    if tfm is None:
        return image, boxes
    transformed = tfm(image=np.array(image), bboxes=[x+[""] for x in boxes])
    return Image.fromarray(transformed["image"]), [x[:4] for x in transformed["bboxes"]]


In [ ]:
#img = Image.open(data[1]["source"])
#i,b = _augment(img, data[1]["boxes"], tfm=transform)
#i

to do:

* random rescaling
* albumentations

In [ ]:
def _area(b):
    return (b[2]-b[0])*(b[3]-b[1])

In [ ]:
def _crop_boxes(boxes, crop_box, classes, area_thresh=0.5):
    """
    TO DO: implement an area check to see how much of each box is inside 
    the crop
    """
    w_offset = crop_box[0]
    h_offset = crop_box[1]
    w = crop_box[2] - crop_box[0]
    h = crop_box[3] - crop_box[1]
    shifted_boxes = [[int(b[0]-w_offset), int(b[1]-h_offset), 
             int(b[2]-w_offset), int(b[3]-h_offset)] for b in boxes]
    # clip boxes to outside of crop area
    clipped_boxes = [[min(max(0, b[0]), w),
                      min(max(0, b[1]), h),
                      min(max(0, b[2]), w),
                      min(max(0, b[3]), h)
    ] for b in shifted_boxes]
    # only keep boxes/labels if they meet the area thresh after clipping
    outboxes = []
    outlabels = []
    for s, c, l in zip(boxes, clipped_boxes, classes):
        if _area(c)/_area(s) >= area_thresh:
            outboxes.append(c)
            outlabels.append(l)
    
    return outboxes, outlabels

In [ ]:
def _random_crop(source, boxes=[], classes=[], resize_to=(1000, 750), cropsize=(200,150), 
                 rand_scale=None, tfm=None):
    """
    uniform random crop
    """
    # load image
    img = Image.open(source)
    if rand_scale is not None:
        s = np.random.uniform(rand_scale[0], rand_scale[1])
        if resize_to is None:
            resize_to = img.size
        resize_to = (int(resize_to[0]/s), int(resize_to[1]/s))
    # augment if albumentation object was passed
    img, boxes = _augment(img, boxes, tfm)
    
    w,h = img.size
    w_ratio = resize_to[0]/w
    h_ratio = resize_to[1]/h
    # resize image and rescale boxes
    if resize_to is not None:
        img = img.resize((resize_to[0],resize_to[1]))
        boxes = [[int(b[0]*w_ratio), int(b[1]*h_ratio), int(b[2]*w_ratio), int(b[3]*h_ratio)] for b in boxes]
    
    # randomly choose an offset and turn into a bounding box
    w_offset = np.random.randint(0, resize_to[0]-cropsize[0])
    h_offset = np.random.randint(0, resize_to[1]-cropsize[1])
    crop_box = [w_offset, h_offset, w_offset+cropsize[0], h_offset+cropsize[1]]
    # crop image and boxes
    img_c = img.crop(crop_box)
    outboxes, outlabels = _crop_boxes(boxes, crop_box, classes)
    return img_c, outboxes, outlabels

In [ ]:
def _crop_around_random_box(source, boxes=[], classes=[], resize_to=(1000, 750), cropsize=(200,150), 
                            rand_scale=None, tfm=None):
    """
    pick a box and crop around it. if no boxes, fall back to _random_crop
    """
    # no boxes? just crop randomly
    if len(boxes) == 0:
        return _random_crop(source, boxes, classes, resize_to, cropsize, tfm=tfm)
    
    # load the image
    img = Image.open(source)
    
    if rand_scale is not None:
        s = np.random.uniform(rand_scale[0], rand_scale[1])
        if resize_to is None:
            resize_to = img.size
        resize_to = (int(resize_to[0]/s), int(resize_to[1]/s))
    
    # augment if albumentation object was passed
    img, boxes = _augment(img, boxes, tfm)
    
    w,h = img.size
    w_ratio = resize_to[0]/w
    h_ratio = resize_to[1]/h
    # resize image and boxes
    if resize_to is not None:
        img = img.resize((resize_to[0],resize_to[1]))
        boxes = [[int(b[0]*w_ratio), int(b[1]*h_ratio), int(b[2]*w_ratio), int(b[3]*h_ratio)] for b in boxes]
    
    # pick a box to crop around
    boxchoice = np.random.randint(0, len(boxes))
    b = boxes[boxchoice]
    # choose a crop box that includes the centroid of the box
    center_x = 0.5*(b[0]+b[2])
    center_y = 0.5*(b[1]+b[3])
    # ok, this merits some explanation: try to sample around the box without going outside
    # the image:
    min_w = max(center_x - cropsize[0],0)
    max_w = min(center_x, resize_to[0]-cropsize[0])
    if min_w < max_w:
        w_offset = np.random.randint(min_w, max_w)
    # if that won't work, pick a crop that goes outside the image
    else:
        w_offset = np.random.randint(center_x-cropsize[0], center_x)
    # now repeat for vertical offset
    min_h = max(center_y - cropsize[1],0)
    max_h = min(center_y, resize_to[1]-cropsize[1])
    if min_h < max_h:
        h_offset = np.random.randint(min_h, max_h)
    else:
        h_offset = np.random.randint(center_y-cropsize[1], center_y)
        
    #w_offset = np.random.randint(max(center_x - cropsize[0],0), min(center_x, resize_to[0]-cropsize[0]))
    #h_offset = np.random.randint(max(center_y - cropsize[1],0), min(center_y, resize_to[1]-cropsize[1]))
    
    crop_box = [w_offset, h_offset, w_offset+cropsize[0], h_offset+cropsize[1]]
    # crop image and boxes
    img_c = img.crop(crop_box)
    outboxes, outlabels = _crop_boxes(boxes, crop_box, classes)
    return img_c, outboxes, outlabels

In [ ]:
# [[2480, 1280, 2720, 1840], [640, 1840, 920, 2080]]
i = 1
img, boxes, labels = _random_crop(data[i]["source"], boxes=data[i]["boxes"], tfm=transform)
img, boxes, labels = _crop_around_random_box(data[i]["source"], boxes=data[i]["boxes"], tfm=transform)
img

In [ ]:
def build_mosaic(sources, resize_each_to=None, outsize=None, minfrac=0.33, 
                  rand_scale=None, tfm=None):
    """
    Construct a mosaic-augmented training example
    
    :sources: list of 4 dictionaries containing box/label info
    :resize_each_to: if not None, a tuple of 2 numbers to resize image to before cropping
    :outsize: dimensions of output image. if None, use resize_each_to
    :minfrac: minimum fraction of a box's area to keep it
    :rand_scale: None or a tuple of 2 numbers; randomly rescale images within that range
    :tfm: albumentations transformation to apply
    """
    np.random.shuffle(sources)
    assert len(sources) == 4
    if outsize is None:
        outsize = resize_each_to
    
    # choose the x and y coordinates of the image splits
    split_x = int(outsize[0]*np.random.uniform(minfrac, 1-minfrac))
    split_y = int(outsize[1]*np.random.uniform(minfrac, 1-minfrac))
    
    # upper left
    crop_size = (split_x, split_y)
    s = sources[0]
    img_ul, boxes_ul, labels_ul = _crop_around_random_box(s["source"], boxes=s["boxes"], 
                                                          classes=s["class"], resize_to=resize_each_to, 
                                                          cropsize=crop_size, rand_scale=rand_scale,
                                                          tfm=tfm)
    # upper right
    crop_size = (outsize[0] - split_x, split_y)
    s = sources[1]
    img_ur, boxes_ur, labels_ur = _crop_around_random_box(s["source"], boxes=s["boxes"], 
                                                          classes=s["class"], resize_to=resize_each_to, 
                                                          cropsize=crop_size, rand_scale=rand_scale,
                                                          tfm=tfm)
    boxes_ur = [[b[0]+split_x, b[1], b[2]+split_x, b[3]] for b in boxes_ur]
    
    # lower left
    crop_size = (split_x, outsize[1] - split_y)
    s = sources[2]
    img_ll, boxes_ll, labels_ll = _crop_around_random_box(s["source"], boxes=s["boxes"], 
                                                          classes=s["class"], resize_to=resize_each_to, 
                                                          cropsize=crop_size, rand_scale=rand_scale,
                                                          tfm=tfm)
    boxes_ll = [[b[0], b[1]+split_y, b[2], b[3]+split_y] for b in boxes_ll]
    
    # lower right
    crop_size = (outsize[0] - split_x, outsize[1]-split_y)
    s = sources[3]
    img_lr, boxes_lr, labels_lr = _crop_around_random_box(s["source"], boxes=s["boxes"], 
                                                          classes=s["class"], resize_to=resize_each_to, 
                                                          cropsize=crop_size, rand_scale=rand_scale,
                                                          tfm=tfm)
    boxes_lr = [[b[0]+split_x, b[1]+split_y, b[2]+split_x, b[3]+split_y] for b in boxes_lr]
    
    boxes = boxes_ul + boxes_ur + boxes_ll + boxes_lr
    labels = labels_ul + labels_ur + labels_ll + labels_lr
    
    img_arr = np.concatenate([
        np.concatenate([np.array(img_ul), np.array(img_ur)], 1),
        np.concatenate([np.array(img_ll), np.array(img_lr)], 1)
    ], 0)
    return Image.fromarray(img_arr), boxes, labels

In [ ]:
img, boxes, classes = build_mosaic([data[0], data[0], data[1], data[1]], rand_scale=(0.25, 2), outsize=(1000, 750),
                                   tfm=transform)
img

In [ ]:
for i in range(min(len(boxes),9)):
    plt.subplot(3,3,i+1)
    plt.imshow(img.crop(boxes[i]))
    plt.title(classes[i])
    plt.axis(False);

In [ ]:
img, boxes, classes = build_mosaic([data[0], data[0], data[1], data[1]], rand_scale=(0.25, 2), 
                                   resize_each_to=(1000, 750),
                                   tfm=transform)
img